<a href="https://colab.research.google.com/github/sthanikan2000/CS3121-Project/blob/main/DS_Proj_employee_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# Read the CSV file into a DataFrame
chatterbox = pd.read_csv('employees.csv')

In [ ]:
# chatterbox.info()

In [ ]:
# Replace "0000" with NaN in 'Year_of_Birth'
chatterbox['Year_of_Birth'] = chatterbox['Year_of_Birth'].replace("'0000'", '0000')
chatterbox['Year_of_Birth'] = chatterbox['Year_of_Birth'].astype('int64')
chatterbox['Year_of_Birth'] = chatterbox['Year_of_Birth'].replace(0, np.nan)

In [ ]:
(chatterbox["Status"]=="Inactive").sum()

764

In [ ]:
chatterbox['Inactive_Date'].isin(['\\N', '0000-00-00']).sum()

233

In [ ]:
((chatterbox['Status'] == 'Inactive') & ((chatterbox['Date_Resigned'].isin(['\\N', '0000-00-00'])) | (chatterbox['Inactive_Date'].isin(['\\N', '0000-00-00'])))).sum()

223

In [ ]:
((chatterbox['Status'] == 'Inactive')&(chatterbox['Inactive_Date'].isin(['\\N', '0000-00-00']))).sum()

0

In [ ]:
(((~chatterbox['Inactive_Date'].isin(['\\N', '0000-00-00']))&( chatterbox['Inactive_Date']!=chatterbox['Date_Resigned']))&(~chatterbox['Date_Resigned'].isin(['\\N', '0000-00-00']))).sum()

6

In [ ]:
((chatterbox["Status"]=="Inactive")&(~chatterbox['Date_Resigned'].isin(['\\N', '0000-00-00']))).sum()

541

In [ ]:
chatterbox[( chatterbox['Inactive_Date']==chatterbox['Date_Resigned']) & chatterbox['Date_Resigned'].isin(['\\N', '0000-00-00']) & chatterbox["Status"]=="Inactive"]

,Employee_No,Employee_Code,Name,Title,Gender,Religion_ID,Marital_Status,Designation_ID,Date_Joined,Date_Resigned,Status,Inactive_Date,Reporting_emp_1,Reporting_emp_2,Employment_Category,Employment_Type,Religion,Designation,Year_of_Birth


In [ ]:
# Impute Date_resigned
mask = (chatterbox['Status'] == 'Inactive') & (chatterbox['Date_Resigned'].isin(['\\N', '0000-00-00'])) & (~chatterbox['Inactive_Date'].isin(['\\N', '0000-00-00']))
chatterbox.loc[mask, 'Date_Resigned'] = chatterbox.loc[mask, 'Inactive_Date']

# Replace '\\N' and '0000-00-00' with NaN
chatterbox['Date_Resigned'].replace(['\\N', '0000-00-00'], np.nan, inplace=True)
chatterbox['Inactive_Date'].replace(['\\N', '0000-00-00'], np.nan, inplace=True)
chatterbox['Reporting_emp_1'].replace(['\\N'], np.nan, inplace=True)
chatterbox['Reporting_emp_2'].replace(['\\N'], np.nan, inplace=True)


In [ ]:
mask = (chatterbox["Gender"]=="Female")&(chatterbox["Title"]=="Mr")

# Since "Ms" is a title used for addressing a woman without specifying her marital status. It can be used for both married and unmarried women.
chatterbox.loc[mask, "Title"] = "Ms"

In [ ]:
mask = (chatterbox["Gender"]=="Male")&(chatterbox["Title"].isin(["Ms","Miss"]))

# Since "Mr" is a title used for addressing a man without specifying his marital status. It can be used for both married and unmarried men.
chatterbox.loc[mask, "Title"] = "Mr"

In [ ]:
mask = (chatterbox["Title"].isin(["Miss"]))&(chatterbox["Marital_Status"]=="Married")
chatterbox.loc[mask, "Marital_Status"] = "Single"

In [ ]:
# Parse columns to datetime format to get all  date times in same format
for i in ["Date_Joined","Date_Resigned","Inactive_Date"]:
  chatterbox[i] = pd.to_datetime(chatterbox[i])

In [ ]:
chatterbox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 997 entries, 0 to 996
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Employee_No          997 non-null    int64         
 1   Employee_Code        997 non-null    int64         
 2   Name                 997 non-null    object        
 3   Title                997 non-null    object        
 4   Gender               997 non-null    object        
 5   Religion_ID          997 non-null    int64         
 6   Marital_Status       930 non-null    object        
 7   Designation_ID       997 non-null    int64         
 8   Date_Joined          997 non-null    datetime64[ns]
 9   Date_Resigned        764 non-null    datetime64[ns]
 10  Status               997 non-null    object        
 11  Inactive_Date        764 non-null    datetime64[ns]
 12  Reporting_emp_1      61 non-null     object        
 13  Reporting_emp_2      0 non-null    

In [ ]:
chatterbox.nunique()

Employee_No            997
Employee_Code          997
Name                   997
Title                    3
Gender                   2
Religion_ID              4
Marital_Status           2
Designation_ID         129
Date_Joined            549
Date_Resigned          350
Status                   2
Inactive_Date          350
Reporting_emp_1         23
Reporting_emp_2          0
Employment_Category      3
Employment_Type          2
Religion                 4
Designation            129
Year_of_Birth           59
dtype: int64

In [ ]:
chatterbox[(chatterbox["Date_Resigned"]!=chatterbox["Inactive_Date"])& ~(chatterbox["Date_Resigned"].isnull())]

,Employee_No,Employee_Code,Name,Title,Gender,Religion_ID,Marital_Status,Designation_ID,Date_Joined,Date_Resigned,Status,Inactive_Date,Reporting_emp_1,Reporting_emp_2,Employment_Category,Employment_Type,Religion,Designation,Year_of_Birth
99,514,3553,Shea Terry MD,Ms,Female,1,Single,65,2017-11-01,2021-01-18,Inactive,2021-01-25,NaN,NaN,Staff,Contarct Basis,Buddhist,Design Engineer,1994.0
117,550,4882,Dr Adrien Beahan,Mr,Male,1,Married,24,2019-09-01,2022-12-31,Inactive,2022-12-27,NaN,NaN,Staff,Contarct Basis,Buddhist,Driver,1973.0
149,601,4943,Luna Harris DDS,Ms,Female,1,Single,48,2020-01-27,2021-11-26,Inactive,2021-11-30,1538,NaN,Staff,Contarct Basis,Buddhist,Draughtsman,1996.0
155,608,4931,Stone Hyatt I,Mr,Male,3,Single,22,2019-12-10,2021-03-14,Inactive,2021-02-14,NaN,NaN,Staff,Contarct Basis,Hindu,Technical Officer,1994.0
243,719,5056,Emilio Lebsack,Mr,Male,1,Married,31,2020-02-02,2021-03-30,Inactive,2021-01-15,NaN,NaN,Labour,Contarct Basis,Buddhist,Unskilled Labor,1958.0
971,2601,5416,Miss Alene Stokes DVM,Mr,Male,1,Married,90,2022-04-28,2022-08-27,Inactive,2022-08-26,NaN,NaN,Staff,Contarct Basis,Buddhist,Assistant manager - HR,1987.0


In [ ]:
chatterbox.drop(["Employee_Code","Name","Reporting_emp_1","Reporting_emp_2","Religion_ID","Designation_ID","Inactive_Date"], axis=1, inplace=True)

In [ ]:
chatterbox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 997 entries, 0 to 996
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Employee_No          997 non-null    int64         
 1   Title                997 non-null    object        
 2   Gender               997 non-null    object        
 3   Marital_Status       930 non-null    object        
 4   Date_Joined          997 non-null    datetime64[ns]
 5   Date_Resigned        764 non-null    datetime64[ns]
 6   Status               997 non-null    object        
 7   Employment_Category  997 non-null    object        
 8   Employment_Type      997 non-null    object        
 9   Religion             997 non-null    object        
 10  Designation          997 non-null    object        
 11  Year_of_Birth        894 non-null    float64       
dtypes: datetime64[ns](2), float64(1), int64(1), object(8)
memory usage: 93.6+ KB


In [ ]:
chatterbox

,Employee_No,Title,Gender,Marital_Status,Date_Joined,Date_Resigned,Status,Employment_Category,Employment_Type,Religion,Designation,Year_of_Birth
0,347,Mr,Male,Married,1993-12-08,NaT,Active,Staff,Permanant,Buddhist,Driver,1965.0
1,348,Mr,Male,Married,1995-03-14,NaT,Active,Staff,Permanant,Buddhist,Driver,1973.0
2,349,Mr,Male,Married,1988-01-27,2021-06-28,Inactive,Staff,Permanant,Buddhist,Account Clerk,1974.0
3,351,Ms,Female,Married,1999-10-01,2022-01-31,Inactive,Staff,Permanant,Catholic,Purchasing Officer,1974.0
4,352,Mr,Male,Married,2001-01-26,NaT,Active,Staff,Permanant,Buddhist,Store Keeper,1980.0
...,...,...,...,...,...,...,...,...,...,...,...,...
992,2836,Ms,Female,Single,2022-11-01,2022-12-22,Inactive,Staff,Contarct Basis,Buddhist,Account Executive,1993.0
993,2890,Mr,Male,Single,2022-11-21,NaT,Active,Staff,Contarct Basis,Buddhist,Quantity Surveyor,1994.0
994,2972,Mr,Male,Single,2022-12-05,NaT,Active,Staff,Contarct Basis,Buddhist,Software Developer,1997.0
995,2973,Mr,Male,Single,2022-12-08,NaT,Active,Staff,Contarct Basis,Buddhist,Quantity Surveyor,1990.0


In [ ]:
from scipy.stats import chi2_contingency

c_n=["Title","Gender","Marital"]
# Assuming 'df' is your DataFrame with two categorical variables
observed = pd.crosstab(chatterbox['Title'], chatterbox['Marital_Status'])
print(observed)
# Perform chi-square test
chi2, p_value, _, _ = chi2_contingency(observed)

print("Chi-square statistic:", chi2)
print("P-value:", p_value)

Marital_Status  Married  Single
Title                          
Miss                  0       6
Mr                  590     277
Ms                   23      34
Chi-square statistic: 29.943033220501565
P-value: 3.14740731935932e-07


In [ ]:
df = chatterbox[["Employee_No", 'Religion', "Title", "Gender", "Status", "Employment_Category", "Employment_Type", "Designation", "Marital_Status", "Year_of_Birth"]].copy()
df['Year_Joined'] = chatterbox['Date_Joined'].dt.year

# Label encoding the categorical nominal variables
le = LabelEncoder()

for i in ['Religion', "Title", "Gender", "Status", "Employment_Category", "Employment_Type", "Designation", "Marital_Status"]:
  if i == "Marital_Status":
    df[i + '_Encoded'] = le.fit_transform(df[i].astype(str))
    # Replace null values with a specific value, such as -1, to represent missing values
    df[i + '_Encoded'].fillna(-1, inplace=True)
  else:
    df[i + '_Encoded'] = le.fit_transform(df[i])
df = df.drop(['Religion', "Title", "Gender", "Status", "Employment_Category", "Employment_Type", "Designation","Marital_Status"], axis=1)


In [ ]:
mask= chatterbox["Marital_Status"].isnull()
df.loc[mask, "Marital_Status_Encoded"] = -1

In [ ]:
mask = (df["Marital_Status_Encoded"]== -1) & (df["Year_of_Birth"].isnull())
mask_temp=mask
df_copy=df.copy()
df=df.drop(df[mask].index)
mask.sum()

9

In [ ]:
df_copy

,Employee_No,Year_of_Birth,Year_Joined,Religion_Encoded,Title_Encoded,Gender_Encoded,Status_Encoded,Employment_Category_Encoded,Employment_Type_Encoded,Designation_Encoded,Marital_Status_Encoded
0,347,1965.0,1993,0,1,1,0,2,1,31,0
1,348,1973.0,1995,0,1,1,0,2,1,31,0
2,349,1974.0,1988,0,1,1,1,2,1,0,0
3,351,1974.0,1999,1,2,0,1,2,1,85,0
4,352,1980.0,2001,0,1,1,0,2,1,108,0
...,...,...,...,...,...,...,...,...,...,...,...
992,2836,1993.0,2022,0,2,0,1,2,0,1,1
993,2890,1994.0,2022,0,1,1,0,2,0,89,1
994,2972,1997.0,2022,0,1,1,0,2,0,106,1
995,2973,1990.0,2022,0,1,1,0,2,0,89,1


In [ ]:
df["Marital_Status_Encoded"].unique()

array([ 0,  1, -1])

In [ ]:
df["Year_of_Birth"].unique()

array([1965., 1973., 1974., 1980., 1962., 1969., 1981., 1967., 1976.,
       1986., 1975., 1984., 1963., 1990., 1961., 1982., 1991., 1978.,
       1994., 1989., 1960., 1993., 1988., 1972., 1995., 1992., 1987.,
         nan, 1997., 1985., 1947., 1966., 1955., 1959., 1957., 1958.,
       1970., 2001., 1996., 1998., 1983., 1999., 1971., 1946., 1952.,
       1968., 1954., 1964., 2000., 1951., 2003., 1949., 1979., 1977.,
       1956., 1953., 2005., 2002., 1950., 2006.])

In [ ]:
### Impute Year of Birth

# Step 1: Split the DataFrame into two sets
df_missing = df[df['Year_of_Birth'].isnull()]
df_not_missing = df[~df['Year_of_Birth'].isnull()]

# Step 2: Create the feature matrix X
X = df_not_missing[['Year_Joined', 'Employment_Category_Encoded', 'Employment_Type_Encoded', 'Gender_Encoded', 'Marital_Status_Encoded']]

# Step 3: Create the target array y
y = df_not_missing['Year_of_Birth']

# Step 4: Train a Random Forest Regressor model
model = RandomForestRegressor()
model.fit(X, y)

# Step 5: Predict missing values in df_missing
X_missing = df_missing[['Year_Joined', 'Employment_Category_Encoded', 'Employment_Type_Encoded', 'Gender_Encoded', 'Marital_Status_Encoded']]
predicted_values = model.predict(X_missing)

# Assign the predicted values to the missing entries in 'Year_of_Birth'
df.loc[df['Year_of_Birth'].isnull(), 'Year_of_Birth'] = predicted_values

In [ ]:
df

,Employee_No,Year_of_Birth,Year_Joined,Religion_Encoded,Title_Encoded,Gender_Encoded,Status_Encoded,Employment_Category_Encoded,Employment_Type_Encoded,Designation_Encoded,Marital_Status_Encoded
0,347,1965.0,1993,0,1,1,0,2,1,31,0
1,348,1973.0,1995,0,1,1,0,2,1,31,0
2,349,1974.0,1988,0,1,1,1,2,1,0,0
3,351,1974.0,1999,1,2,0,1,2,1,85,0
4,352,1980.0,2001,0,1,1,0,2,1,108,0
...,...,...,...,...,...,...,...,...,...,...,...
992,2836,1993.0,2022,0,2,0,1,2,0,1,1
993,2890,1994.0,2022,0,1,1,0,2,0,89,1
994,2972,1997.0,2022,0,1,1,0,2,0,106,1
995,2973,1990.0,2022,0,1,1,0,2,0,89,1


In [ ]:
df["Marital_Status_Encoded"].unique()

array([ 0,  1, -1])

In [ ]:
# Step 1: Split the DataFrame into two sets
df_missing = df[df['Marital_Status_Encoded']==-1]
df_not_missing = df[~(df['Marital_Status_Encoded']==-1)]

# Step 2: Create the feature matrix X
X = df_not_missing[['Year_Joined', 'Employment_Category_Encoded', 'Employment_Type_Encoded', 'Gender_Encoded', 'Year_of_Birth', 'Title_Encoded']]

# Step 3: Create the target array y
y = df_not_missing['Marital_Status_Encoded']

# Step 4: Train a Random Forest Classifier model
model = RandomForestClassifier()
model.fit(X, y)

# Step 5: Predict missing values in df_missing
X_missing = df_missing[['Year_Joined', 'Employment_Category_Encoded', 'Employment_Type_Encoded', 'Gender_Encoded', 'Year_of_Birth', 'Title_Encoded']]
predicted_values = model.predict(X_missing)

# Assign the predicted values to the missing entries in 'Marital_Status_Encoded'
df.loc[df['Marital_Status_Encoded']==-1, 'Marital_Status_Encoded'] = predicted_values
df.loc[df['Marital_Status_Encoded']==0, 'Marital_Status'] = "Married"
df.loc[df['Marital_Status_Encoded']==1, 'Marital_Status'] = "Single"

In [ ]:
df["Marital_Status"].unique()

array(['Married', 'Single'], dtype=object)

In [ ]:
mask = chatterbox["Marital_Status"].isnull() & chatterbox["Year_of_Birth"].isnull()
print(mask.sum())

chatterbox_missing=chatterbox.drop(chatterbox[~(mask)].index)
chatterbox=chatterbox.drop(chatterbox[mask].index)
chatterbox["Marital_Status"]=df["Marital_Status"]
chatterbox["Year_of_Birth"]=df["Year_of_Birth"]

chatterbox = pd.concat([chatterbox, chatterbox_missing])
chatterbox["Marital_Status"] =chatterbox["Marital_Status"].fillna(chatterbox["Marital_Status"].mode()[0])
chatterbox["Year_of_Birth"] =chatterbox["Year_of_Birth"].fillna(chatterbox["Year_of_Birth"].mean())

chatterbox['Year_of_Birth'] = chatterbox['Year_of_Birth'].astype(int)

# Order the DataFrame by Employee_No
chatterbox = chatterbox.sort_values(by='Employee_No')


9


In [ ]:
x=chatterbox["Marital_Status"].mode()[0]
x

'Married'

In [ ]:
chatterbox.isnull().sum()

Employee_No              0
Title                    0
Gender                   0
Marital_Status           0
Date_Joined              0
Date_Resigned          233
Status                   0
Employment_Category      0
Employment_Type          0
Religion                 0
Designation              0
Year_of_Birth            0
dtype: int64

In [ ]:
chatterbox

,Employee_No,Title,Gender,Marital_Status,Date_Joined,Date_Resigned,Status,Employment_Category,Employment_Type,Religion,Designation,Year_of_Birth
0,347,Mr,Male,Married,1993-12-08,NaT,Active,Staff,Permanant,Buddhist,Driver,1965
1,348,Mr,Male,Married,1995-03-14,NaT,Active,Staff,Permanant,Buddhist,Driver,1973
2,349,Mr,Male,Married,1988-01-27,2021-06-28,Inactive,Staff,Permanant,Buddhist,Account Clerk,1974
3,351,Ms,Female,Married,1999-10-01,2022-01-31,Inactive,Staff,Permanant,Catholic,Purchasing Officer,1974
4,352,Mr,Male,Married,2001-01-26,NaT,Active,Staff,Permanant,Buddhist,Store Keeper,1980
...,...,...,...,...,...,...,...,...,...,...,...,...
992,2836,Ms,Female,Single,2022-11-01,2022-12-22,Inactive,Staff,Contarct Basis,Buddhist,Account Executive,1993
993,2890,Mr,Male,Single,2022-11-21,NaT,Active,Staff,Contarct Basis,Buddhist,Quantity Surveyor,1994
994,2972,Mr,Male,Single,2022-12-05,NaT,Active,Staff,Contarct Basis,Buddhist,Software Developer,1997
995,2973,Mr,Male,Single,2022-12-08,NaT,Active,Staff,Contarct Basis,Buddhist,Quantity Surveyor,1990


In [ ]:
df1=pd.read_csv("attendance_preprocessed_200635E.csv")
df2=pd.read_csv("leaves_preprocessed_200635E.csv")
df3=pd.read_csv("salary_preprocessed_200635E.csv")

In [ ]:
df2.loc[df2['Type']=="Half Day","Count"]=df2.loc[df2['Type']=="Half Day","Count"]/2
df2.info()
df2.drop(["Type"],axis=1,inplace=True)
df2.groupby('Employee_No').total()
df2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Employee_No  120 non-null    int64  
 1   Type         120 non-null    object 
 2   Count        120 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.9+ KB


AttributeError: ignored

In [ ]:
merged_df = pd.merge(chatterbox, df1, on='Employee_No', how='left')
print(merged_df.shape)
merged_df = pd.merge(merged_df, df2, on='Employee_No', how='left')
print(merged_df.shape)
merged_df = pd.merge(merged_df, df3, on='Employee_No', how='left')

In [ ]:
merged_df

In [ ]:
merged_df["Hourly_Time"].isnull().sum()

In [ ]:
merged_df["Type"].isnull().sum()

In [ ]:
merged_df["Earnings"].isnull().sum()

In [ ]:
merged_df["Count"].isnull().sum()

In [ ]:
merged_df["Hourly_Time"].isnull().sum()

In [ ]:
chatterbox.duplicated(subset=chatterbox.columns.difference(["Employee_No",	"Employee_Code"	,"Name"])).sum()